# calibration

In [ ]:
import cv2
import os

cap = cv2.VideoCapture(0)
os.makedirs("calib_images", exist_ok=True)
count = 0

print("📸 Show the printed chessboard at different angles and distances.")
print("Press SPACE to capture, or 'q' to quit.\n")

while True:
    ret, frame = cap.read()
    if not ret:
        break
    cv2.imshow("Calibration Capture", frame)
    key = cv2.waitKey(1) & 0xFF

    if key == ord(' '):
        filename = f"calib_images/img_{count:02d}.jpg"
        cv2.imwrite(filename, frame)
        print(f"✅ Saved {filename}")
        count += 1
    elif key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


📸 Show the printed chessboard at different angles and distances.
Press SPACE to capture, or 'q' to quit.



In [ ]:
import cv2
import numpy as np
import glob

chessboard_size = (9, 6)
square_size = 1

objp = np.zeros((chessboard_size[0]*chessboard_size[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2)
objp *= square_size

objpoints, imgpoints = [], []
images = glob.glob('calib_images/*.jpg')

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, chessboard_size, None)
    if ret:
        corners2 = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1),
                                    (cv2.TermCriteria_EPS + cv2.TermCriteria_MAX_ITER, 30, 0.001))
        objpoints.append(objp)
        imgpoints.append(corners2)
        cv2.drawChessboardCorners(img, chessboard_size, corners2, ret)
        cv2.imshow('img', img)
        cv2.waitKey(100)

cv2.destroyAllWindows()
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
np.savez('calibration_data.npz', camera_matrix=mtx, dist_coeffs=dist)
print("✅ Calibration saved to calibration_data.npz")
print("Camera matrix:\n", mtx)
print("Distortion coefficients:\n", dist)


✅ Calibration saved to calibration_data.npz
Camera matrix:
 [[487.70504965   0.         328.8415351 ]
 [  0.         490.49036099 240.18432311]
 [  0.           0.           1.        ]]
Distortion coefficients:
 [[ 0.05259666  0.09826273 -0.01883954 -0.01101908 -0.36870366]]


# code

In [1]:
# --------------------------------------------------------------
# 6D POSE ESTIMATION USING ArUco MARKERS  (OpenCV 4.12+ Compatible)
# --------------------------------------------------------------
import cv2
import numpy as np

# --- Load calibration ---
with np.load('calibration_data.npz') as X:
    camera_matrix, dist_coeffs = [X[i] for i in ('camera_matrix', 'dist_coeffs')]

# --- ArUco detector (new API) ---
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
parameters = cv2.aruco.DetectorParameters()
detector = cv2.aruco.ArucoDetector(aruco_dict, parameters)

marker_length = 0.05  #5cm

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("❌ Webcam not found")

print("🎯 Pose estimation running… press 'q' to quit.")

# 3-D model points of marker corners (centered at marker origin)
half = marker_length / 2
objp = np.array([
    [-half,  half, 0],
    [ half,  half, 0],
    [ half, -half, 0],
    [-half, -half, 0]
], dtype=np.float32)

while True:
    ok, frame = cap.read()
    if not ok:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    corners, ids, _ = detector.detectMarkers(gray)

    if ids is not None:
        cv2.aruco.drawDetectedMarkers(frame, corners, ids)

        for marker_corners in corners:
            # reshape for solvePnP
            imgp = marker_corners[0].astype(np.float32)
            success, rvec, tvec = cv2.solvePnP(objp, imgp, camera_matrix, dist_coeffs)
            if success:
                cv2.drawFrameAxes(frame, camera_matrix, dist_coeffs, rvec, tvec, 0.03)

    cv2.imshow("6D Pose Estimation – Lomash Robotics", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


🎯 Pose estimation running… press 'q' to quit.


In [2]:
# --------------------------------------------------------------
# AUGMENTED REALITY 3-D CUBE OVERLAY USING ArUco MARKERS (OpenCV 4.12 +)
# Lomash Robotics – ABV-IIITM Gwalior
# --------------------------------------------------------------
import cv2
import numpy as np

# --- Load calibration file you generated earlier ---
with np.load("calibration_data.npz") as X:
    camera_matrix, dist_coeffs = [X[i] for i in ("camera_matrix", "dist_coeffs")]

# --- ArUco detector (new API) ---
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
parameters = cv2.aruco.DetectorParameters()
detector = cv2.aruco.ArucoDetector(aruco_dict, parameters)

marker_length = 0.05  # 5 cm

# 3-D coordinates of the marker corners in its local frame
half = marker_length / 2
objp = np.array([
    [-half,  half, 0],
    [ half,  half, 0],
    [ half, -half, 0],
    [-half, -half, 0]
], dtype=np.float32)

# Define cube vertices in 3-D (same size as marker)
cube = np.array([
    [-half, -half, 0], [ half, -half, 0],
    [ half,  half, 0], [-half,  half, 0],
    [-half, -half, marker_length], [ half, -half, marker_length],
    [ half,  half, marker_length], [-half,  half, marker_length]
], dtype=np.float32)

edges = [
    (0,1),(1,2),(2,3),(3,0),
    (4,5),(5,6),(6,7),(7,4),
    (0,4),(1,5),(2,6),(3,7)
]

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("❌ Webcam not found")

print("🧊 AR overlay running – press 'q' to quit.")

while True:
    ok, frame = cap.read()
    if not ok:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    corners, ids, _ = detector.detectMarkers(gray)

    if ids is not None:
        cv2.aruco.drawDetectedMarkers(frame, corners, ids)

        for marker_corners in corners:
            imgp = marker_corners[0].astype(np.float32)
            success, rvec, tvec = cv2.solvePnP(objp, imgp, camera_matrix, dist_coeffs)
            if success:
                # Project cube vertices to image
                imgpts, _ = cv2.projectPoints(cube, rvec, tvec, camera_matrix, dist_coeffs)
                imgpts = np.int32(imgpts).reshape(-1, 2)

                # Draw cube
                for (a, b) in edges:
                    cv2.line(frame, tuple(imgpts[a]), tuple(imgpts[b]), (0,255,0), 2)
                top = imgpts[4:8]
                cv2.drawContours(frame, [top], -1, (255,0,0), -1)
                cv2.addWeighted(frame, 0.8, frame, 0.2, 0, frame)

    cv2.imshow("AR 3-D Cube Overlay – Lomash Robotics", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


🧊 AR overlay running – press 'q' to quit.
